In [1]:
import pandas as pd
from backtesting import Backtest
from library import Client, Payload, ConcreateStrategy
from datetime import datetime
import numpy as np
pd.options.display.float_format = "{:6.2f}".format

client = Client()

/usr/local/lib/python3.11/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [6]:
class My_Strategy(ConcreateStrategy):
    def init(self):
        self.use_atr()
        self.use_donchian_channel()
        self.use_half_donchian_channel()
        self.use_sma()

    def next(self):
        # SMAによる手仕舞い
        if (self.is_price_lower_than_sma() and self.position.is_long) or (
            self.is_price_higher_than_sma() and self.position.is_short
        ):
            self.position.close()
            return

        # DCによる手仕舞い
        if (self.is_half_donchian_channel_updated_highest() and self.position.is_short) or (
            self.is_half_donchian_channel_updated_lowest() and self.position.is_long
        ):
            self.position.close()
            return

        # 買い注文
        if (
            self.is_donchian_channel_updated_highest()
            and self.is_price_higher_than_sma()
            and not self.position.is_long
        ):
            self.buy_with_atr()
            return

In [16]:
bt = Backtest(
    client.get_in_cache(Payload(
        tickers="spy",
        start=datetime.strptime("2016-01-01", "%Y-%m-%d"),
        end=datetime.strptime("2023-01-01", "%Y-%m-%d"))
    ),
    My_Strategy,
    cash=10000000,
    commission=0.003,
    exclusive_orders=True,
)

# 出力
# output = bt.run()
# print(output)
# bt.plot()

optimize = bt.optimize(
    maximize="SQN",
    prop_donchian_channel_period = range(100, 200, 20),
    prop_atr_stop_loss = np.arange(4.0, 7.0, 1).tolist(),
    prop_atr_take_profit = np.arange(3.0, 5.0, 1).tolist(),
    prop_sma_period = range(160, 170, 10),
)
bt.plot()
print(optimize)
print(optimize._strategy)

Start                     2010-01-04 00:00:00
End                       2021-12-31 00:00:00
Duration                   4379 days 00:00:00
Exposure Time [%]                       55.88
Equity Final [$]                  15677778.36
Equity Peak [$]                   15820440.13
Return [%]                              56.78
Buy & Hold Return [%]                  319.09
Return (Ann.) [%]                        3.82
Volatility (Ann.) [%]                    8.60
Sharpe Ratio                             0.44
Sortino Ratio                            0.63
Calmar Ratio                             0.29
Max. Drawdown [%]                      -13.23
Avg. Drawdown [%]                       -1.60
Max. Drawdown Duration     1218 days 00:00:00
Avg. Drawdown Duration       58 days 00:00:00
# Trades                                   71
Win Rate [%]                            73.24
Best Trade [%]                           6.00
Worst Trade [%]                         -5.13
Avg. Trade [%]                    

In [14]:
print( range(160, 170, 10)[1]
       )

IndexError: range object index out of range